## Spark streaming

In [13]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [5]:
spark = SparkSession.builder.config("spark.sql.streaming.schemaInference", True).getOrCreate()

stream = spark.\
    readStream.\
    format("ws").\
    option("schema", "ticker").\
    load() # we need to pass `option("schema", "ticker")` to get correct channel subscribed

query = stream.select("side", "product_id", "last_size", "best_bid", "best_ask", "time").\
    writeStream.\
    format("console").\
    outputMode("append").\
    option("truncate", "false").\
    start()

query.awaitTermination(10) # Let's wait for 10 seconds.
query.stop() # Let's stop the query
stream.printSchema()
#spark.stop() # And stop the whole session

root
 |-- type: string (nullable = false)
 |-- trade_id: long (nullable = false)
 |-- sequence: long (nullable = false)
 |-- time: timestamp (nullable = false)
 |-- product_id: string (nullable = false)
 |-- price: double (nullable = false)
 |-- side: string (nullable = false)
 |-- last_size: double (nullable = false)
 |-- best_bid: double (nullable = false)
 |-- best_ask: double (nullable = false)



In [7]:
#Example of output in terminal
# Batch: 4
# -------------------------------------------
# +----+----------+----------+--------+--------+-----------------------+
# |side|product_id|last_size |best_bid|best_ask|time                   |
# +----+----------+----------+--------+--------+-----------------------+
# |sell|ETH-USD   |0.62424001|3199.2  |3199.23 |2025-11-13 22:19:04.936|
# |sell|ETH-USD   |0.178     |3199.12 |3199.23 |2025-11-13 22:19:04.936|
# +----+----------+----------+--------+--------+-----------------------+

Uruchamiająć `stream.start()` uruchamiamy w osobnym demonie websocket który streamuje wyniki. Jeżeli wystąpi jakiś błąd po stronie front-endu (np. błąd parsowania kolejnej linijki Pythona) fakt ten nie zostanie zgłoszony do sparka i socket pozostanie otwarty! Należy pamiętać, by zamykać stream za każdym razem używająć metody `stop()` (w powyższym przykładzie `query.stop()`). W przypadku utracenia referencji do zapytania, należy zastopować całą sesję również metodą `stop()` (w powyższym przykładzie `spark.stop()`) 

In [7]:
# Panic button - press only if you messed up opening new websocket and lost reference to it

# query.stop()
# spark.stop()

# Zadanie 1

**Analiza strumienia danych CoinBase (2p)**. Napisz zapytanie, które wypisuje średnią wartość wybranego parametru (np. `price`) w przesuwnych oknach czasowych względem czasu transakcji (kolumna `time`), grupując po relacji wymiany (z jakiej waluty na jaką walutę - kolumna `product_id`). 

In [3]:
# window parameters
windowSize = "240"
slideSize = "60"
windowDuration = '{} seconds'.format(windowSize)
slideDuration = '{} seconds'.format(slideSize)

spark = SparkSession\
    .builder\
    .config("spark.sql.streaming.schemaInference", True)\
    .getOrCreate()

stream = spark\
    .readStream\
    .format("ws")\
    .option("schema", "ticker")\
    .load() 

query = stream\
    .select("product_id", "price", "time")\
    .groupBy(F.window("time", windowDuration, slideDuration), "product_id").mean("price")\
    .orderBy("window")

query = query.writeStream\
    .format("console")\
    .outputMode("complete")\
    .option("truncate", "false")\
    .start()

query.awaitTermination(30) 
query.stop()

```bash
-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|ETH-USD   |3223.962056737588  |
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|ETH-BTC   |0.03327200000000001|
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|BTC-EUR   |83385.39285714286  |
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|BTC-USD   |96857.95134715026  |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|ETH-USD   |3223.962056737588  |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|ETH-BTC   |0.03327200000000001|
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|BTC-USD   |96857.95134715026  |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|BTC-EUR   |83385.39285714286  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|BTC-USD   |96857.95134715026  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|ETH-BTC   |0.03327200000000001|
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|BTC-EUR   |83385.39285714286  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|ETH-USD   |3223.962056737588  |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|BTC-EUR   |83385.39285714286  |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|BTC-USD   |96857.95134715026  |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|ETH-BTC   |0.03327200000000001|
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|ETH-USD   |3223.962056737588  |
+------------------------------------------+----------+-------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|ETH-USD   |3225.512130681818  |
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|ETH-BTC   |0.03328571428571429|
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|BTC-USD   |96858.05940251572  |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|ETH-USD   |3225.512130681818  |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|ETH-BTC   |0.03328571428571429|
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|BTC-USD   |96858.05940251572  |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|BTC-USD   |96858.05940251572  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|ETH-BTC   |0.03328571428571429|
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|ETH-USD   |3225.512130681818  |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|BTC-USD   |96858.05940251572  |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|ETH-BTC   |0.03328571428571429|
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|ETH-USD   |3225.512130681818  |
+------------------------------------------+----------+-------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|ETH-USD   |3225.682375979112  |
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|ETH-BTC   |0.03328571428571429|
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:23:00, 2025-11-14 16:27:00}|BTC-USD   |96858.45993883793  |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|ETH-USD   |3225.7723042505586 |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|ETH-BTC   |0.03328869565217392|
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|BTC-USD   |96856.9985678392   |
|{2025-11-14 16:24:00, 2025-11-14 16:28:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|BTC-USD   |96856.9985678392   |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|ETH-BTC   |0.03328869565217392|
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:25:00, 2025-11-14 16:29:00}|ETH-USD   |3225.7723042505586 |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|BTC-EUR   |83387.09999999999  |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|BTC-USD   |96856.9985678392   |
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|ETH-BTC   |0.03328869565217392|
|{2025-11-14 16:26:00, 2025-11-14 16:30:00}|ETH-USD   |3225.7723042505586 |
|{2025-11-14 16:27:00, 2025-11-14 16:31:00}|ETH-USD   |3226.31046875      |
|{2025-11-14 16:27:00, 2025-11-14 16:31:00}|BTC-USD   |96850.26802816901  |
|{2025-11-14 16:27:00, 2025-11-14 16:31:00}|ETH-BTC   |0.03332            |
+------------------------------------------+----------+-------------------+
```

# Zadanie 2

**Watermarking i dane opóźnione (2p).** 
Zmodyfikuj zapytanie z zadania 1 tak, aby zademonstować mechanizm znaków wodnych (watermarks) i obsługi danych opóźnionych. W konsoli powinno być widać, że aktualizują się odpowiednie wiersze tabeli wynikowej (tryb update), w szczególności aktualizacja wcześniejszych okien czasowych po przybyciu danych opóźnionych. **Do rozwiązania tego zadania proszę dołączyć przykładowy output i jego opis wyjaśniający na konkretnym przykładzie działanie znaku wodnego i danych opóźnionych**. 

Do ćwiczenia można wykorzystać skrypt w katalogu `/mock` napisany w [Scala-cli](https://scala-cli.virtuslab.org), który posłuży jako kontrolowane źródło danych CoinBase przez Websocket. 

Skrypt można uruchomić wykorzystując Docker:

```
make image
make run
```

Spowoduje to utworzenie websocketowego serwera pod adresem `ws://mock:8025`

Po uruchomieniu serwera należy wykonać poniższą komórkę, w której zapytanie czyta dane z utworzonego websocketa. Skrypt wysyła przykładowe wiadomości w formacie CoinBase co 10 sekund:

- W pierwszej serii wysyłane wiadomości o znacznikach czasowych 0s, 14s, 7s  
- W drugiej serii wysyłane są wiadomości o znacznikach czasowych 15s, 8s, 21s  
- W trzeciej serii wysyłane są wiadomości o znacznikach czasowych 4s, 17s  

Dla tych danych można ustawić okno czasowe na interwał 10 sekund. Skrypt można też zmodyfikować, tak aby wysyłał inne dane. 


In [6]:
windowSize = "10"
slideSize = "5"
watermarkSize = "10"
awaitTermination = 40

windowDuration = '{} seconds'.format(windowSize)
slideDuration = '{} seconds'.format(slideSize)
watermarkDuration = '{} seconds'.format(watermarkSize)

spark = SparkSession\
    .builder\
    .config("spark.sql.streaming.schemaInference", True)\
    .getOrCreate()

# Note url option!
# we pass explicit url option to subscribe to our mock service

stream = spark\
    .readStream\
    .format("ws")\
    .option("schema", "ticker")\
    .option("url", "ws://mock:8025")\
    .load() 

query = stream\
    .select("product_id", "price", "time")\
    .withWatermark("time", watermarkDuration)\
    .groupBy(F.window("time", windowDuration, slideDuration), "product_id").mean("price")\

query = query.writeStream\
    .format("console")\
    .outputMode("update")\
    .option("truncate", "false")\
    .start()

query.awaitTermination(awaitTermination) 
query.stop()


```bash
Batch: 0 
-------------------------------------------
+------+----------+----------+ --> no updates
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+

Batch: 1
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:05, 2021-11-01 00:00:15}|ETH-USD   |199.06575282217665| 
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |192.52398919068668| 
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |29.070320572008978| 
|{2021-10-31 23:59:55, 2021-11-01 00:00:05}|ETH-USD   |15.986793309029078| 
+------------------------------------------+----------+------------------+
Batch: 2 
-------------------------------------------
+------+----------+----------+ --> no updates
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+

Batch: 3
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:05, 2021-11-01 00:00:15}|ETH-USD   |431.54909261101074| 
|{2021-11-01 00:00:20, 2021-11-01 00:00:30}|ETH-USD   |393.2874923837869 | 
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |427.18791685668407| 
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |375.4339774700433 | 
|{2021-11-01 00:00:15, 2021-11-01 00:00:25}|ETH-USD   |557.5425633759323 | 
+------------------------------------------+----------+------------------+

Batch: 4
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+

Batch: 5
-------------------------------------------
+------------------------------------------+----------+-----------------+
|window                                    |product_id|avg(price)       |
+------------------------------------------+----------+-----------------+
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |483.508401707395 | 
|{2021-11-01 00:00:15, 2021-11-01 00:00:25}|ETH-USD   |604.9141256446543|
+------------------------------------------+----------+-----------------+

```

#### **Batch: 0**

no updates

#### **Batch: 1**

Windows:

2021-10-31 23:59:55, 2021-11-01 00:00:05 **0s, 4s timestamps updates**

2021-11-01 00:00:00, 2021-11-01 00:00:10 **7s, 8s timestamps updates**

2021-11-01 00:00:05, 2021-11-01 00:00:15 **14s, 15s timestamps updates**

2021-11-01 00:00:10, 2021-11-01 00:00:20 **17s, 21s timestamps updates**

2021-11-01 00:00:15, 2021-11-01 00:00:25 **no updates**

2021-11-01 00:00:20, 2021-11-01 00:00:30 **no updates**


#### **Batch: 2** 

no updates

#### **Batch: 3**

Windows:

2021-10-31 23:59:55, 2021-11-01 00:00:05 **no updates**

2021-11-01 00:00:00, 2021-11-01 00:00:10 **no updates**

2021-11-01 00:00:05, 2021-11-01 00:00:15 **14s, 15s timestamps updates**

2021-11-01 00:00:10, 2021-11-01 00:00:20 **17s, 21s timestamps updates**

2021-11-01 00:00:15, 2021-11-01 00:00:25 **21s timestamps updates**

2021-11-01 00:00:20, 2021-11-01 00:00:30 **21s timestamps updates**

#### **Batch: 4**

no updates

#### **Batch: 5**

Windows:

2021-10-31 23:59:55, 2021-11-01 00:00:05 **no updates**

2021-11-01 00:00:00, 2021-11-01 00:00:10 **no updates**

2021-11-01 00:00:05, 2021-11-01 00:00:15 **no updates**

2021-11-01 00:00:10, 2021-11-01 00:00:20 **17s, 21s timestamps updates**

2021-11-01 00:00:15, 2021-11-01 00:00:25 **21s timestamps updates**

2021-11-01 00:00:20, 2021-11-01 00:00:30 **no updates**

# Zadanie 3

**Łączenie strumieni (1p)**. Korzystając z łączenia strumieni połącz dane z kanału `ticker` (transakcje kupna `side="buy"`) razem z danymi o transakcjach napływających co sekundę `heartbeat` korzystając z `trade_id` i odpowiedniego id w kanale heartbeat. Wypisz połączony strumień danych.

Na moment tworzenia zadania 15.11.2023 kanał `heartbeat` zwraca błędne dane o dacie (np. `1970-01-04 13:53:57.645339`). Połączenie z kanałem `ticker` pozwala uzyskać poprawne informacje. Cóż za wspaniałe zastosowanie joina!


In [8]:
spark = SparkSession.builder.config("spark.sql.streaming.schemaInference", True).getOrCreate()

stream = spark.\
    readStream.\
    format("ws").\
    option("schema", "heartbeat").\
    load() 

stream.printSchema()

root
 |-- type: string (nullable = false)
 |-- sequence: long (nullable = false)
 |-- last_trade_id: long (nullable = false)
 |-- product_id: string (nullable = false)
 |-- time: timestamp (nullable = false)



In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    config("spark.sql.streaming.schemaInference", True).\
    config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True).\
    getOrCreate()  

stream_ticker = spark.\
    readStream.\
    format("ws").\
    option("schema", "ticker").\
    load()

stream_heartbeat = spark.\
    readStream.\
    format("ws").\
    option("schema", "heartbeat").\
    load()

buy = stream_ticker\
    .select("side", "product_id", "price", "time")\
    .filter(stream_ticker.side == "buy")\
    .withColumnRenamed("side", "buy_side") \
    .withColumnRenamed("product_id", "buy_product_id") \
    .withColumnRenamed("price", "buy_price") \
    .withColumnRenamed("time", "buy_time")

hearbeat = stream_heartbeat\
    .select("last_trade_id", "product_id", "sequence", "time", "type")

buy_heartbeat= buy.join(hearbeat, F.expr(
            """
            product_id = buy_product_id
            """
            ))
    
query = buy_heartbeat.writeStream\
    .format("console")\
    .outputMode("append")\
    .option("truncate", "false")\
    .start()

query.awaitTermination(awaitTermination) 
query.stop()

```bash
-------------------------------------------
Batch: 0
-------------------------------------------
+--------+--------------+---------+--------+-------------+----------+--------+----+----+
|buy_side|buy_product_id|buy_price|buy_time|last_trade_id|product_id|sequence|time|type|
+--------+--------------+---------+--------+-------------+----------+--------+----+----+
+--------+--------------+---------+--------+-------------+----------+--------+----+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
|buy_side|buy_product_id|buy_price|buy_time               |last_trade_id|product_id|sequence   |time                      |type     |
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.395|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.507|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.362|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82308.26 |2025-11-14 19:57:55.523|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.39 |100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.322|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.391|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.398|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.98 |2025-11-14 19:57:54.679|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.322|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.391|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.471|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.98 |2025-11-14 19:57:54.694|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.394|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286737238|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.395|100626051    |BTC-EUR   |45286737238|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.507|100626051    |BTC-EUR   |45286737238|1970-01-04 13:53:57.645339|heartbeat|
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
only showing top 20 rows

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
|buy_side|buy_product_id|buy_price|buy_time               |last_trade_id|product_id|sequence   |time                      |type     |
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.395|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.507|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.362|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82308.26 |2025-11-14 19:57:55.523|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.39 |100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.322|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.391|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.398|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.98 |2025-11-14 19:57:54.679|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.322|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.391|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.471|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.98 |2025-11-14 19:57:54.694|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.394|100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.358|100626051    |BTC-EUR   |45286740242|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.395|100626051    |BTC-EUR   |45286740242|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82304.99 |2025-11-14 19:57:55.507|100626051    |BTC-EUR   |45286740242|1970-01-04 13:53:57.645339|heartbeat|
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
only showing top 20 rows

-------------------------------------------
Batch: 3
-------------------------------------------
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
|buy_side|buy_product_id|buy_price|buy_time               |last_trade_id|product_id|sequence   |time                      |type     |
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286737238|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626034    |BTC-EUR   |45286735157|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286736020|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626034    |BTC-EUR   |45286734611|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626036    |BTC-EUR   |45286735516|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286737667|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626034    |BTC-EUR   |45286734889|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286736425|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286737014|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286739056|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286740242|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286740965|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286738128|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286740685|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286738637|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286739646|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.22 |2025-11-14 19:58:10.95 |100626051    |BTC-EUR   |45286741507|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.21 |2025-11-14 19:58:10.918|100626051    |BTC-EUR   |45286736733|1970-01-04 13:53:57.645339|heartbeat|
|buy     |BTC-EUR       |82338.21 |2025-11-14 19:58:10.918|100626051    |BTC-EUR   |45286737238|1970-01-04 13:53:57.645339|heartbeat|
+--------+--------------+---------+-----------------------+-------------+----------+-----------+--------------------------+---------+
only showing top 20 rows
```
